# Working with Spans

In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
repo = Path().cwd().parent.parent.resolve()
sys.path.append(str(repo))

In [2]:
from pprint import pformat
from pprint import pprint

from examples.expositional.end2end_apps.custom_app.custom_app import CustomApp
from examples.expositional.end2end_apps.custom_app.custom_retriever import CustomRetriever
import pandas as pd

from trulens_eval import instruments
from trulens_eval.trace.category import Categorizer
from trulens_eval.tru_custom_app import TruCustomApp

In [3]:
from trulens_eval import Tru
Tru().reset_database()
Tru().start_dashboard(_dev=repo, force=True)

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.
Force stopping dashboard ...
Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://10.0.0.25:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [4]:
# Create custom app:
ca = CustomApp(delay=0.0, alloc=0)

# Create trulens wrapper:
ta = TruCustomApp(
    ca,
    app_id="customapp",
)

In [5]:
instruments.Instrument().print_instrumentation()

Module examples.expositional.end2end_apps.custom_app.custom_app*
  Class examples.expositional.end2end_apps.custom_app.custom_app.CustomApp
    Method get_context: (self, input: str)
    Method respond_to_query: (self, input: str)
    Method arespond_to_query: (self, input: str)
  Class examples.expositional.end2end_apps.custom_app.custom_app.CustomTemplate
    Method fill: (self, question, answer)

Module examples.expositional.end2end_apps.custom_app.custom_llm*
  Class examples.expositional.end2end_apps.custom_app.custom_llm.CustomLLM
    Method generate: (self, prompt: str, temperature: Optional[float] = None) -> str
      Span type: SpanType.LLM (static), spanner: <function CustomLLM.set_llm_span at 0x2abacc360>
      Span type: SpanType.LLM (id: 11463407760=?11463407760), spanner: <function CustomLLM.set_llm_span at 0x2abacc360>

Module examples.expositional.end2end_apps.custom_app.custom_memory*
  Class examples.expositional.end2end_apps.custom_app.custom_memory.CustomMemory
    

In [6]:
ta.print_instrumented()

Components:
	TruCustomApp (Other) at 0x2abe93ac0 with path __app__
	CustomApp (Custom) at 0x2aba99490 with path __app__.app
	CustomLLM (Custom) at 0x2ab45b890 with path __app__.app.llm
	CustomMemory (Custom) at 0x2abf66d10 with path __app__.app.memory
	CustomReranker (Custom) at 0x2abf70790 with path __app__.app.reranker
	CustomRetriever (Custom) at 0x2abf67f10 with path __app__.app.retriever
	CustomTemplate (Custom) at 0x2abf66750 with path __app__.app.template
	CustomTool (Custom) at 0x2abd27dd0 with path __app__.app.tools[0]
	CustomTool (Custom) at 0x107dad610 with path __app__.app.tools[1]
	CustomTool (Custom) at 0x2abf703d0 with path __app__.app.tools[2]

Methods:
Object at 0x2abf66d10:
	<function CustomMemory.remember at 0x2abacc720> with path __app__.app.memory
Object at 0x2abf67f10:
	<function CustomRetriever.retrieve_chunks at 0x2abacd080> with path __app__.app.retriever
Object at 0x2ab45b890:
	<function CustomLLM.generate at 0x107dc0cc0> with path __app__.app.llm
Object at 0x

In [7]:
with ta as recorder:
    res = ca.respond_to_query(f"hello")

rec = recorder.get()

In [8]:
rec.calls[0].model_dump()

{'stack': [{'path': 'app',
   'method': {'obj': {'cls': {'name': 'CustomApp',
      'module': {'package_name': 'examples.expositional.end2end_apps.custom_app',
       'module_name': 'examples.expositional.end2end_apps.custom_app.custom_app'},
      'bases': None},
     'id': 11469952144,
     'init_bindings': None},
    'name': 'respond_to_query'}},
  {'path': 'app',
   'method': {'obj': {'cls': {'name': 'CustomApp',
      'module': {'package_name': 'examples.expositional.end2end_apps.custom_app',
       'module_name': 'examples.expositional.end2end_apps.custom_app.custom_app'},
      'bases': None},
     'id': 11469952144,
     'init_bindings': None},
    'name': 'get_context'}},
  {'path': 'app.retriever',
   'method': {'obj': {'cls': {'name': 'CustomRetriever',
      'module': {'package_name': 'examples.expositional.end2end_apps.custom_app',
       'module_name': 'examples.expositional.end2end_apps.custom_app.custom_retriever'},
      'bases': None},
     'id': 11474992912,
     'in

In [9]:
spans = Categorizer.spans_of_record(rec)

pd.DataFrame(
    [(
        s.trace_id & 0xff,
        s.name,
        type(s),
        s.span_type,
        s.span_id & 0xff,
        s.parent_span_id & 0xff if s.parent_span_id else 0,
        s.attributes
    ) for s in spans],
    columns=[
        "trace_id",
        "name",
        "type",
        "span_type",
        "span_id",
        "parent_span_id",
        "attributes"
    ],
)


,trace_id,name,type,span_type,span_id,parent_span_id,attributes
0,121,root,<class 'trulens_eval.trace.span.SpanRoot'>,SpanRoot,204,0,"{'trulens_eval@span_type': 'SpanRoot', 'trulen..."
1,121,retrieve_chunks,<class 'trulens_eval.trace.span.SpanRetriever'>,SpanRetriever,56,179,"{'trulens_eval@span_type': 'SpanRetriever', 't..."
2,121,rerank,<class 'trulens_eval.trace.span.SpanReranker'>,SpanReranker,197,179,"{'trulens_eval@span_type': 'SpanReranker', 'tr..."
3,121,invoke,<class 'trulens_eval.trace.span.SpanTool'>,SpanTool,214,179,"{'trulens_eval@span_type': 'SpanTool', 'trulen..."
4,121,invoke,<class 'trulens_eval.trace.span.SpanTool'>,SpanTool,40,179,"{'trulens_eval@span_type': 'SpanTool', 'trulen..."
5,121,get_context,<class 'trulens_eval.trace.span.SpanOther'>,SpanOther,179,14,"{'trulens_eval@span_type': 'SpanOther', 'trule..."
6,121,remember,<class 'trulens_eval.trace.span.SpanMemory'>,SpanMemory,184,14,"{'trulens_eval@span_type': 'SpanMemory', 'trul..."
7,121,generate,<class 'trulens_eval.trace.span.SpanLLM'>,SpanLLM,78,14,"{'trulens_eval@span_type': 'SpanLLM', 'trulens..."
8,121,fill,<class 'trulens_eval.trace.span.SpanOther'>,SpanOther,134,14,"{'trulens_eval@span_type': 'SpanOther', 'trule..."
9,121,remember,<class 'trulens_eval.trace.span.SpanMemory'>,SpanMemory,85,14,"{'trulens_eval@span_type': 'SpanMemory', 'trul..."


In [10]:
for span in spans:
    pprint(span)
    pprint(span.model_dump())
    print()

SpanRoot(name='root', kind=<SpanKind.INTERNAL: 0>, status=<StatusCode.OK: 1>, status_description=None, start_timestamp=1714442346605224000, end_timestamp=1714417146452397000, context=HashableSpanContext(trace_id=0x00000000000000003dadf42d4b535e79, span_id=0xc461bfb604e9f8cc, trace_flags=0x00, trace_state=[], is_remote=False), events=[], links={}, attributes={'trulens_eval@span_type': 'SpanRoot', 'trulens_eval@record_id': 'record_hash_c7c1852d78e621164a9d2965e2e5686c'}, attributes_metadata={}, record=Record(record_id='record_hash_c7c1852d78e621164a9d2965e2e5686c', app_id='customapp', cost=Cost(n_requests=0, n_successful_requests=0, n_classes=0, n_tokens=0, n_stream_chunks=0, n_prompt_tokens=0, n_completion_tokens=0, cost=0.0), perf=Perf(start_time=datetime.datetime(2024, 4, 29, 18, 59, 5, 465477), end_time=datetime.datetime(2024, 4, 29, 18, 59, 6, 452397)), ts=datetime.datetime(2024, 4, 29, 18, 59, 6, 452457), tags='-', meta=None, main_input='hello', main_output="The answer to hello is 

/opt/homebrew/Caskroom/miniconda/base/envs/py311_trulens/lib/python3.11/site-packages/pydantic/main.py:314: UserWarning: Pydantic serializer warnings:
  Expected `Union[str, bool, int, float, json-or-python[json=list[str], python=list[str]], json-or-python[json=list[bool], python=list[bool]], json-or-python[json=list[int], python=list[int]], json-or-python[json=list[float], python=list[float]]]` but got `list` - serialized value may not be as expected
  Expected `Union[str, bool, int, float, json-or-python[json=list[str], python=list[str]], json-or-python[json=list[bool], python=list[bool]], json-or-python[json=list[int], python=list[int]], json-or-python[json=list[float], python=list[float]]]` but got `dict` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
/opt/homebrew/Caskroom/miniconda/base/envs/py311_trulens/lib/python3.11/site-packages/pydantic/main.py:314: UserWarning: Pydantic serializer warnings:
  Expected `Union[str, bool, int, float